# AI/ML 프로세스 매뉴얼(5) - SageMaker Endpoint 생성가이드

## 1) Yolov5 clone 및 필요 라이브러리 설치

In [2]:
!git clone https://github.com/ultralytics/yolov5
%cd /home/ec2-user/SageMaker/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 14391, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 14391 (delta 3), reused 7 (delta 3), pack-reused 14379
Receiving objects: 100% (14391/14391), 13.33 MiB | 20.58 MiB/s, done.
Resolving deltas: 100% (9958/9958), done.
/home/ec2-user/SageMaker/yolov5
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     |████████████████████████████████| 881.9 MB 2.4 kB/s             █████▊| 873.8 MB 83.9 MB/s eta 0:00:01
     |████████████████████████████████| 23.3 MB 74.5 MB/s            
     |████████████████████████████████| 78 kB 856 kB/s             
     |████████████████████████████████| 881.9 MB 2.3 kB/s             
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.63.0
    Uninstalling tqdm-4.63.0:
      Successfully uninstalled tqdm-4.63.0


In [6]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


## 2) 모델 프레임워크 변환 (Pytorch -> Tensorflow)
* 사전조건
    * 이전에 학습 된 모델 가중치 (ex_ best.pt)가 있어야 함

In [ ]:
!python export.py --weights /home/ec2-user/SageMaker/best.pt --include saved_model --nms

## 3) 모델 패키징
* 아래의 폴더구조 생성 후 변환 된 모델 패키징
* s3 버킷에 패키징 된 모델 저장

In [ ]:
%cd /home/ec2-user/SageMaker
!mkdir export && mkdir export/Servo
!mv best_saved_model export/Servo/1
!tar -czvf model.tar.gz export/

In [ ]:
!aws s3 cp model.tar.gz "s3://posco-test-20221011/model.tar.gz"

## 4) SageMaker Endpoint 생성

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import backend
from sagemaker.tensorflow import TensorFlowModel
from sagemaker import get_excution_role

In [ ]:
model_data = "s3://posco-test-20221011/model.tar.gz" ## 패키징 된 모델의 S3 위치
model = TensorFlowModel(model_data=model_data, ## 모델 데이터 위치
                        framework_version='2.8', ## 프레임워크 버전
                        role=get_execution_role()) ## SageMaker 실행 역할
INSTANCE_TYPE = 'ml.c6i.xlarge' ## Endpoint 실행 인스턴스 타입
ENDPOINT_NAME = 'yolov5-demo' ## Endpoint 이름

predictor = model.deploy(initial_instance_count=1, ## Endpoint 실행 인스턴스 수
                         instance_type=INSTANCE_TYPE,
                         endpoint_name=ENDPOINT_NAME)

## 5) 배포된 모델 확인 및 실시간 추론체크

In [ ]:
import numpy as np
import json
import boto3, botocore

config = botocore.config.Config(read_time = 80)
runtime = boto3.client('runtime.sagemaker', config = config)
ENDPOINT_NAME = 'yolov5-demo' ## 실행할 Endpoint 이름

modelHeight, modelWidth = 640, 640 ## 이미지 사이즈
blank_image = np.zeros((modelHeight,modelWidth,3), np.uint8) ## 공백 이미지 생성
data = np.array(blank_image.astype(np.float32)/255.) ## 이미지 스케일링
payload = json.dumps([data.tolist()]) ## 이미지 데이터 형 변환

response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME, ## 실시간 추론 테스트
                                   ContentType='application/json',
                                   Body=payload)

result = json.loads(response['Body'].read().decode()) ## 결과 출력
print('Results: ', result)